In [1]:
import serial.tools.list_ports
ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p)

COM8 - USB Serial Port (COM8)
COM10 - USB-SERIAL CH340 (COM10)


In [2]:
import serial
ser = serial.Serial('COM10', 115200, timeout=10)

Implement and validate the Echo design on the FPGA.

You can use the following script to send a sequence of 12 bytes using Python:

In [18]:
ser.write(b'ABCD EFGHI J')

12

In [19]:
print(ser.read(12))

b'ABCD EFGHI J'


UPDATE! A common error in the implementation of UART RX is to keep it active for more than the duration of a UART frame, that is, more than 10*CLKS_PER_BIT.

In the above script, Python is sending bytes immediately after each other. So if your UART RX is not back in sIDLE at the correct time, it may miss the start of the next byte being transmitted.

This leads to an error in which the string that your receive in Python has the first character correct ('A'), but the remaining are erroneous (typically something like '\xa1').

To confirm whether this is the problem, you can test the Python code below which sends the bytes one by one, with a waiting in between:

In [11]:
import time 
tst_str = 'ABCD EFGHI J'
tst_arr = str.encode(tst_str)

for j in range(len(tst_arr)):
    ser.write(tst_arr[j:j + 1])
    time.sleep(0.001)

print(ser.read(12))

b'\xffABCD EFGHI '


In [15]:
ser.close()

This code snippet creates two random operands (A,B) of length OPERAND_WIDTH

In [5]:
import random
import time

OPERAND_WIDTH = 512

N_BYTES = int(OPERAND_WIDTH/8)

A = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)
B = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)

res = A + B

print("A     = ", hex(A))
print("B     = ", hex(B))
print("A + B = ", hex(res))

A_bytes = bytearray.fromhex(format(A, 'x'))
B_bytes = bytearray.fromhex(format(B, 'x'))

A     =  0xc425bf2f1818e9fbd583377b95cab1393c5cb2a9df9320f1efbfaa742e009f1ca07a2211abb85c90d81e691611bf0c7393f5403f548ba3f9f84e3219bc7ae7e2
B     =  0xfb55fb0a2c11f62d004ee43b039416a6ab96abbc8d4082e9a56fd7f9d084de73fe3406fa0a8239f799ceec0df5faf12249da63cbbc5db164eec919557c0a4b5a
A + B =  0x1bf7bba39442ae028d5d21bb6995ec7dfe7f35e666cd3a3db952f826dfe857d909eae290bb63a968871ed552407b9fd95ddcfa40b10e9555ee7174b6f3885333c


This code snippet sends the two operands (A,B) via the serial port, byte by byte. It then receives the result of the addition (length N_BYTES+1) and informs whether the received value matches the expected result.

In [6]:
for op_byte in A_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

for op_byte in B_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

res_rcvd = ser.read(N_BYTES+1) 

res2 = int.from_bytes(res_rcvd, "big")

if res2==res:
    print("Result received correctly!")
    print("Expected   = ", hex(res))
    print("Received   = ", hex(res2))
else:
    print("Result INCORRECT!")
    print("Expected   = ", hex(res))
    print("Received   = ", hex(res2))

Result received correctly!
Expected   =  0x1bf7bba39442ae028d5d21bb6995ec7dfe7f35e666cd3a3db952f826dfe857d909eae290bb63a968871ed552407b9fd95ddcfa40b10e9555ee7174b6f3885333c
Received   =  0x1bf7bba39442ae028d5d21bb6995ec7dfe7f35e666cd3a3db952f826dfe857d909eae290bb63a968871ed552407b9fd95ddcfa40b10e9555ee7174b6f3885333c
